In [1]:
import matplotlib.pyplot as plt
import os
import pylab as pl
from sklearn.metrics import precision_recall_curve
from itertools import cycle
import numpy as np



In [2]:
!pip install pycocotools==2.0.0
!pip install torchvision==0.4.0
!pip uninstall -y protobuf
!pip install protobuf==3.9.0

You are using pip version 9.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Uninstalling protobuf-3.9.0:
  Successfully uninstalled protobuf-3.9.0
You are using pip version 9.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Error processing line 1 of /home/ma-user/anaconda3/envs/Pytorch-1.0.0/lib/python3.6/site-packages/protobuf-3.5.1-py3.6-nspkg.pth:

  Traceback (most recent call last):
    File "/home/ma-user/anaconda3/envs/Pytorch-1.0.0/lib/python3.6/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 568, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
    10

In [3]:
import tools._init_paths

%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorboardX as tb
from datasets.factory import get_imdb
from model.train_val import get_training_roidb, train_net
from model.config import cfg, cfg_from_file, cfg_from_list, get_output_dir, get_output_tb_dir

In [4]:
#模型训练中的参数设置
imdb_name = "voc_2007_trainval"
imdbval_name = "voc_2007_test"
imdb_name = imdbval_name # 变更为测试验证集合 

# 使用的预训练模型位置
weight = "./data/imagenet_weights/vgg16.pth"
# 迭代次数
max_iters = 100
# cfg模型文件位置
cfg_file = None
set_cfgs = None

In [5]:
#定义加载数据集函数
def combined_roidb(imdb_names):
    
    def get_roidb(imdb_name):
        # 加载数据集
        imdb = get_imdb(imdb_name) #记录了数据集的路径之类的信息,函数通过给数据集的名字返回了该数据集对应的类的对象。
        print('Loaded dataset `{:s}` for training'.format(imdb.name))
        # 使用ground truth作为数据集策略
        #对每张影像名调用_load_pascal_annotation，即可以知道该函数读取的是影像对应存储目标的xml文件。最后返回了“boxes”，“gt_classes”等构成的字典。
        imdb.set_proposal_method(cfg.TRAIN.PROPOSAL_METHOD)
        print('Set proposal method: {:s}'.format(cfg.TRAIN.PROPOSAL_METHOD))
        #增加翻转,对每张影像在上面进行了求最大处理，并记录影像路径还有影像高宽等信息在roidb中并返回，返回的roidb是一个长度与影像数一致的列表
        roidb = get_training_roidb(imdb) 
        return roidb
    
    #对于每个数据集返回了带有该数据集信息的imdb和包含每张影像感兴趣区域信息的roidb（包括每张影像点目标和影像宽高等信息）
    roidbs = [get_roidb(s) for s in imdb_names.split('+')]
    roidb = roidbs[0] #roidb包括数据集所有影像的影像路径，所有影像对应的目标，以及宽高等信息。
    if len(roidbs) > 1:
        for r in roidbs[1:]:
            roidb.extend(r)
        tmp = get_imdb(imdb_names.split('+')[1])
        imdb = datasets.imdb.imdb(imdb_names, tmp.classes)
    else:
        imdb = get_imdb(imdb_names)
    return imdb, roidb

In [6]:
#训练过程
import numpy as np
if cfg_file is not None:
    cfg_from_file(cfg_file)
if set_cfgs is not None:
    cfg_from_list(set_cfgs)

print('Using config:')
print(cfg)

np.random.seed(cfg.RNG_SEED)

# 加载训练数据集
imdb, roidb = combined_roidb(imdb_name)
print('{:d} roidb entries'.format(len(roidb)))

Using config:
{'TRAIN': {'LEARNING_RATE': 0.001, 'MOMENTUM': 0.9, 'WEIGHT_DECAY': 0.0001, 'GAMMA': 0.1, 'STEPSIZE': [30000], 'DISPLAY': 10, 'DOUBLE_BIAS': True, 'TRUNCATED': False, 'BIAS_DECAY': False, 'USE_GT': False, 'ASPECT_GROUPING': False, 'SNAPSHOT_KEPT': 3, 'SUMMARY_INTERVAL': 180, 'SCALES': [600], 'MAX_SIZE': 1000, 'IMS_PER_BATCH': 1, 'BATCH_SIZE': 128, 'FG_FRACTION': 0.25, 'FG_THRESH': 0.5, 'BG_THRESH_HI': 0.5, 'BG_THRESH_LO': 0.1, 'USE_FLIPPED': True, 'BBOX_REG': True, 'BBOX_THRESH': 0.5, 'SNAPSHOT_ITERS': 5000, 'SNAPSHOT_PREFIX': 'res101_faster_rcnn', 'BBOX_NORMALIZE_TARGETS': True, 'BBOX_INSIDE_WEIGHTS': [1.0, 1.0, 1.0, 1.0], 'BBOX_NORMALIZE_TARGETS_PRECOMPUTED': True, 'BBOX_NORMALIZE_MEANS': [0.0, 0.0, 0.0, 0.0], 'BBOX_NORMALIZE_STDS': [0.1, 0.1, 0.2, 0.2], 'PROPOSAL_METHOD': 'gt', 'HAS_RPN': True, 'RPN_POSITIVE_OVERLAP': 0.7, 'RPN_NEGATIVE_OVERLAP': 0.3, 'RPN_CLOBBER_POSITIVES': False, 'RPN_FG_FRACTION': 0.5, 'RPN_BATCHSIZE': 256, 'RPN_NMS_THRESH': 0.7, 'RPN_PRE_NMS_TOP_N

In [7]:
from nets.vgg16 import vgg16
from model.test import im_detect
import torch
import cv2
from torchvision.ops import nms

from modelarts.session import Session
sess = Session()

np.random.seed(cfg.RNG_SEED)
num_images = len(imdb.image_index) 
print('classes & len', imdb.num_classes, num_images)
all_boxes = [[[] for _ in range(num_images)]
            for _ in range(imdb.num_classes)]
outputdir = imdb._get_voc_results_file_template()

net = vgg16()
print('cfg.ANCHOR_SCALES', cfg.ANCHOR_SCALES)

net.create_architecture(imdb.num_classes, tag='default',  anchor_scales=cfg.ANCHOR_SCALES)
saved_model = "./models/vgg16-voc0712/vgg16_faster_rcnn_iter_110000.pth"
# 加载权重文件
net.load_state_dict(torch.load(saved_model, map_location=lambda storage, loc: storage))

net.eval()
# 选择计算设施
net.to(net._device)


for i in range(num_images):
    # filter out any ground truth boxes
    if cfg.TEST.HAS_RPN:
        box_proposals = None
    else:
        box_proposals = roidb[i]['boxes'][roidb[i]['gt_classes'] == 0]

    im = cv2.imread(imdb.image_path_at(i))
    
    scores, boxes = im_detect( net, im)

    CONF_THRESH = 0.7
    NMS_THRESH = 0.3
    
    thresh=0
    max_per_image=100
    cls_dets = {}
    for j in range(1, imdb.num_classes):          #判断检测到的目标框是否满足阈值
        inds = np.where(scores[:, j] > thresh)[0]
        cls_scores = scores[inds, j]
        cls_boxes = boxes[inds, j*4:(j+1)*4]
        cls_dets = np.hstack((cls_boxes, cls_scores[:, np.newaxis])) \
            .astype(np.float32, copy=False)
        keep = nms(torch.from_numpy(cls_boxes), torch.from_numpy(cls_scores), NMS_THRESH)
        cls_dets = cls_dets[keep, :]
        all_boxes[j][i] = cls_dets

    # Limit to max_per_image detections *over all classes*
    if max_per_image > 0:
        image_scores = np.hstack([all_boxes[j][i][:, -1]
                                  for j in range(1, imdb.num_classes)])
        if len(image_scores) > max_per_image:
            image_thresh = np.sort(image_scores)[-max_per_image]
            for j in range(1, imdb.num_classes):
                keep = np.where(all_boxes[j][i][:, -1] >= image_thresh)[0]

print('Evaluating detections',)
imdb._write_voc_results_file(all_boxes)

classes & len 21 4952
cfg.ANCHOR_SCALES [8, 16, 32]
Evaluating detections
Writing aeroplane VOC results file
Writing bicycle VOC results file
Writing bird VOC results file
Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file
Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC results file
Writing horse VOC results file
Writing motorbike VOC results file
Writing person VOC results file
Writing pottedplant VOC results file
Writing sheep VOC results file
Writing sofa VOC results file
Writing train VOC results file
Writing tvmonitor VOC results file


In [8]:
import voc_eval

def do_python_eval(self, output_dir='output'):
    annopath = os.path.join(
        self._devkit_path,
        'VOC' + self._year,
        'Annotations',
        '{:s}.xml')
    imagesetfile = os.path.join(
        self._devkit_path,
        'VOC' + self._year,
        'ImageSets',
        'Main',
        self._image_set + '.txt')
    cachedir = os.path.join(self._devkit_path, 'annotations_cache')
    aps = []
    # The PASCAL VOC metric changed in 2010
    use_07_metric = True if int(self._year) < 2010 else False
    print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    index = 0
    total_mAP = 0
    for i, cls in enumerate(self._classes):
        print('index', index)
        if cls == '__background__':
            print('is __background__')
            continue
        filename = self._get_voc_results_file_template().format(cls)
        rec, prec, ap = voc_eval.voc_eval(filename, annopath, imagesetfile, cls, cachedir, use_07_metric=use_07_metric)
        print('cls', cls)
        print('rec', rec)
        print('prec', prec)
        print('ap', ap)
        total_mAP = total_mAP + ap
        index = index + 1
    mAP = total_mAP / index
    print('mAP：', mAP)
        


In [9]:
do_python_eval(imdb)

VOC07 metric? Yes
index 0
is __background__
index 0
cachefile /home/ma-user/work/data/VOCdevkit2007/VOC2007/ImageSets/Main/test.txt_annots.pkl
detfile /home/ma-user/work/data/VOCdevkit2007/results/VOC2007/Main/comp4_d5ab384b-960c-40ff-928d-460ffd97f710_det_test_aeroplane.txt
cls aeroplane
rec [0.00350877 0.00701754 0.01052632 ... 0.89473684 0.89473684 0.89473684]
prec [1.         1.         1.         ... 0.00177469 0.00177468 0.00177467]
ap 0.7244807182930142
index 1
cachefile /home/ma-user/work/data/VOCdevkit2007/VOC2007/ImageSets/Main/test.txt_annots.pkl
detfile /home/ma-user/work/data/VOCdevkit2007/results/VOC2007/Main/comp4_d5ab384b-960c-40ff-928d-460ffd97f710_det_test_bicycle.txt
cls bicycle
rec [0.00296736 0.00593472 0.00890208 ... 0.89317507 0.89317507 0.89317507]
prec [1.         1.         1.         ... 0.00200759 0.00200758 0.00200756]
ap 0.7252522987215836
index 2
cachefile /home/ma-user/work/data/VOCdevkit2007/VOC2007/ImageSets/Main/test.txt_annots.pkl
detfile /home/ma-us

cls tvmonitor
rec [0.00324675 0.00649351 0.00974026 ... 0.92207792 0.92207792 0.92207792]
prec [1.         1.         1.         ... 0.00202299 0.00202298 0.00202296]
ap 0.7273271395418532
mAP： 0.7019767790843461


In [10]:
print('Evaluating detections')
output_dir = 'output'
imdb.evaluate_detections(all_boxes, output_dir)

Evaluating detections
Writing aeroplane VOC results file
Writing bicycle VOC results file
Writing bird VOC results file
Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file
Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC results file
Writing horse VOC results file
Writing motorbike VOC results file
Writing person VOC results file
Writing pottedplant VOC results file
Writing sheep VOC results file
Writing sofa VOC results file
Writing train VOC results file
Writing tvmonitor VOC results file
VOC07 metric? Yes
AP for aeroplane = 0.7245
AP for bicycle = 0.7253
AP for bird = 0.6983
AP for boat = 0.5522
AP for bottle = 0.6469
AP for bus = 0.7466
AP for car = 0.8322
AP for cat = 0.8391
AP for chair = 0.5464
AP for cow = 0.7788
AP for diningtable = 0.5785
AP for dog = 0.7922
AP for horse = 0.8199
AP for motorbike = 0.7251
AP for perso